In [46]:
import pandas as pd
import numpy as np

#users.csv data cleaning

In [17]:
users_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TDS/Project 1 - Basel10/users_raw.csv')
users_df.head()

,login,name,company,location,email,hireable,bio,public_repos,followers,following,created_at
0,tarsius,Jonas Bernoulli,NaN,"Basel, Switzerland",NaN,NaN,NaN,39,1345,45,2008-09-17T19:18:37Z
1,aalmiray,Andres Almiray,NaN,"Basel, Switzerland",NaN,NaN,I code for fun and help others in the process....,253,887,65,2008-06-17T00:30:56Z
2,marcoroth,Marco Roth,NaN,"Basel, Switzerland",NaN,True,"Rubyist, Full-Stack Devloper and Open Source C...",193,559,1135,2014-01-15T17:12:11Z
3,klmr,Konrad Rudolph,@Roche,"Basel, CH",NaN,NaN,"Geneticist 🧬, computer scientist 𝝺 and softwar...",115,473,109,2008-12-07T20:55:40Z
4,MrNeRF,janusch,NaN,"Basel, Switzerland",NaN,True,Professional C++ Software Engineer. Skills: CU...,34,437,38,2017-11-21T18:17:58Z


In [47]:
# Clean up company names

users_df['company']=users_df['company'].str.strip().str.replace('@','').str.upper()
users_df['company'].value_counts()

,count
company,
,113
ADOBE,20
UNIVERSITY OF BASEL,20
ROCHE,12
NOVARTIS,7
...,...
"WAYFAIR, PREV. UVINUM",1
JOURNALIST & CODER,1
MICKEL TECH,1


In [48]:
users_df.replace(to_replace=np.nan,value='',inplace=True)
users_df.replace(to_replace='TRUE',value='true',inplace=True)
users_df.replace(to_replace='FALSE',value='false',inplace=True)

users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349 entries, 0 to 348
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   login         349 non-null    object
 1   name          349 non-null    object
 2   company       349 non-null    object
 3   location      349 non-null    object
 4   email         349 non-null    object
 5   hireable      349 non-null    object
 6   bio           349 non-null    object
 7   public_repos  349 non-null    int64 
 8   followers     349 non-null    int64 
 9   following     349 non-null    int64 
 10  created_at    349 non-null    object
dtypes: int64(3), object(8)
memory usage: 30.1+ KB


In [24]:
users_df.to_csv('/content/drive/MyDrive/Colab Notebooks/TDS/Project 1 - Basel10/users.csv',index=False)

# users_csv Data Analysis

In [49]:
# top 5 users in Basel by number of followers

users_df.sort_values(by='followers',ascending=False).head()['login'].to_list()

['tarsius', 'aalmiray', 'marcoroth', 'klmr', 'MrNeRF']

In [50]:
#5 earliest registered GitHub users in Basel

users_df.sort_values(by='created_at').head()['login'].to_list()

['bennyzen', 'aalmiray', 'pvillega', 'tarsius', 'amaunz']

In [44]:
#Which company do the majority of these developers work at

users_df['company'].str.split(r'[,/|;&.-]').explode().unique()

array(['', 'ROCHE', 'GENEDATA AG', 'KADOA', 'YOO AG',
       'ELEKTRONISCHES STUDIO BASEL', 'UNIVERSITY OF CAMBRIDGE',
       'FRIEDRICH MIESCHER INSTITUTE FOR BIOMEDICAL RESEARCH (FMI)',
       'PHARMA', 'BIOZENTRUM', ' UNIVERSITY OF BASEL', 'CREADI', 'GARNIX',
       'DHBERN ', ' STADT', 'GESCHICHTE', 'BASEL', 'RED HAT',
       'UNIVERSITY HOSPITAL BASEL', ' RESEARCH DEPARTMENT',
       'RISM DIGITAL CENTER', 'UNDEF', 'EYELEVEL SCHWEIZ AG', 'ERIGON',
       'SUTROORG', 'QUANTUM CONSULTING BY CHRISTENSEN', 'ADOBE',
       ' APACHE', 'IWF', 'WEB', 'ZPFING', 'ADOBE SYSTEMS', 'INTELLATECH',
       'FMI', 'ADFINIS', 'VERNEMQ', 'QUADBIO LAB', ' D', 'BSSE',
       ' ETH ZURICH', 'F', ' HOFFMANN', 'LA ROCHE AG', ' SWITZERLAND',
       'FRIEDRICH MIESCHER INSTITUTE FOR BIOMEDICAL RESEARCH', 'INTEL',
       'TIPI', 'BUILD', 'ARACONWISS GMBH', 'SNAP INC',
       ' UNIVERSITY OF BASEL ', ' SIB SWISS INSTITUTE OF BIOINFORMATICS ',
       ' ELIXIR SWITZERLAND', 'UNIVERSITY OF BASEL', 'QUADBIO',
  

In [33]:
users_df['company'].value_counts()

,count
company,
,113
ADOBE,20
UNIVERSITY OF BASEL,20
ROCHE,12
NOVARTIS,7
...,...
"WAYFAIR, PREV. UVINUM",1
JOURNALIST & CODER,1
MICKEL TECH,1


In [64]:
#Let's define leader_strength as followers / (1 + following). Who are the top 5 in terms of leader_strength?

users_df['leader_strength']=users_df['followers']/(1+users_df['following'])
users_df.sort_values(by='leader_strength',ascending=False).head()['login'].to_list()

['dpryan79', 'wasserth', 'ravage84', 'elanmart', 'quadbiolab']

In [65]:
#What is the correlation between the number of followers and the number of public repositories among users in Basel

users_df[['followers','public_repos']].corr()

,followers,public_repos
followers,1.000000,0.344951
public_repos,0.344951,1.000000


In [22]:
#Does creating more repos help users get more followers? Using regression, estimate how many additional followers a user gets per additional public repository.

from sklearn.linear_model import LinearRegression

X=users_df['public_repos'].values.reshape(-1,1)
y=users_df['followers'].values.reshape(-1,1)

model=LinearRegression()
model.fit(X,y)

print(model.coef_)

[[0.6736902]]


In [45]:
#Average of following per user for hireable=true minus the average following for the rest

users_df.groupby('hireable')['following'].mean()[0]-users_df.groupby('hireable')['following'].mean()[1]

<ipython-input-45-43c55821b273>:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  users_df.groupby('hireable')['following'].mean()[0]-users_df.groupby('hireable')['following'].mean()[1]
<ipython-input-45-43c55821b273>:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  users_df.groupby('hireable')['following'].mean()[0]-users_df.groupby('hireable')['following'].mean()[1]


45.86879432624113

In [25]:
#Regression slope of followers on bio word count

# function to count words

import regex
def uwc(text):
       words = regex.findall(r'\p{L}+', text)  # Find all Unicode word characters
       return len(words)

#apply function to the bio column in the dataframe

users_df['bio_wc']=users_df['bio'].apply(uwc)
users_df[['bio','bio_wc']].head()


,bio,bio_wc
0,,0
1,I code for fun and help others in the process....,23
2,"Rubyist, Full-Stack Devloper and Open Source C...",8
3,"Geneticist 🧬, computer scientist 𝝺 and softwar...",7
4,Professional C++ Software Engineer. Skills: CU...,21


In [26]:
#Regression slope of followers on bio word count

wc_reg=LinearRegression()
wc_reg.fit(users_df['bio_wc'].values.reshape(-1,1),users_df['followers'].values.reshape(-1,1))

print(wc_reg.coef_)

[[2.10903379]]


In [29]:
#[fraction of users with email when hireable=true] minus [fraction of users with email for the rest]

users_df.groupby('hireable')['email'].count()

,email
hireable,
True,67
,282


In [30]:
((users_df.groupby('hireable')['email'].count()[0])-(users_df.groupby('hireable')['email'].count()[1]))/len(users_df['email'])

<ipython-input-30-a4b03435fdb5>:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ((users_df.groupby('hireable')['email'].count()[0])-(users_df.groupby('hireable')['email'].count()[1]))/len(users_df['email'])
<ipython-input-30-a4b03435fdb5>:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ((users_df.groupby('hireable')['email'].count()[0])-(users_df.groupby('hireable')['email'].count()[1]))/len(users_df['email'])


-0.6160458452722063

In [67]:
users_df['last_name']= users_df['name'].str.strip().str.split(' ').str[-1]
users_df['last_name'].drop(index=users_df[users_df['last_name'] == '   '].index, inplace=True) #drop rows where the value in the last_name column is an empty string. inplace=True modifies the existing DataFrame
ln_counts=users_df['last_name'].value_counts()
ln_counts

,count
last_name,
,7
Landolt,2
Tan,2
Group,2
Brand,2
...,...
Rusconi,1
Litvinov,1
Bühler,1


In [68]:
ln_counts_2 = ln_counts[ln_counts == 2].index.to_list()
print(ln_counts_2)

['Landolt', 'Tan', 'Group', 'Brand', 'Guggisberg', 'GmbH', 'Arnold', 'Christensen', 'Roth', 'Fink']


REPOS.csv Analysis

In [51]:
repos_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TDS/Project 1 - Basel10/repositories.csv')
repos_df.head()

,login,full_name,created_at,stargazers_count,watchers_count,language,has_projects,has_wiki,license
0,tarsius,tarsius/notmuch-transient,2024-08-05T16:06:30Z,2,2,Emacs Lisp,False,False,GNU General Public License v3.0
1,tarsius,tarsius/tray,2024-08-11T00:30:12Z,3,3,Emacs Lisp,False,False,GNU General Public License v3.0
2,tarsius,tarsius/llama,2024-08-10T21:22:27Z,2,2,Emacs Lisp,False,False,GNU General Public License v3.0
3,tarsius,tarsius/ol-notmuch,2024-08-05T16:07:55Z,5,5,Emacs Lisp,False,False,GNU General Public License v3.0
4,tarsius,tarsius/notmuch-addr,2024-08-05T16:07:30Z,5,5,Emacs Lisp,False,False,GNU General Public License v3.0


In [55]:
#What are the 3 most popular license among these users?

repos_df['license'].value_counts().head(3).index.to_list()

['MIT License', 'Apache License 2.0', 'Other']

In [62]:
#Which programming language is most popular among these users

repos_df['language'].value_counts().head(3).index.to_list()

['JavaScript', 'Python', 'Java']

In [60]:
#Which programming language is the second most popular among users who joined after 2020

repos_df[repos_df['created_at']>'2020-01-01']['language'].value_counts().head(3).index.to_list()


['Python', 'JavaScript', 'TypeScript']

In [63]:
#Which language has the highest average number of stars per repository

repos_df.groupby('language')['stargazers_count'].mean().sort_values(ascending=False).head(1).index.to_list()

['PureScript']

In [73]:
#Correlation between projects and wiki enabled

repos_df[['has_projects','has_wiki']].corr()

,has_projects,has_wiki
has_projects,1.000000,0.206028
has_wiki,0.206028,1.000000


In [ ]:
#Who created the most repositories on weekends (UTC)? List the top 5 users' login in order, comma-separated

repos_df['created_at']=pd.to_datetime(repos_df['created_at'])
repos_df['created_at'].dt.day_name().value_counts()